In [312]:
from sklearn import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split


device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)
    
    
iris = datasets.load_iris()

In [313]:

X, y = iris.data, iris.target

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [314]:
# PyTorch로 데이터 로딩
train_data = TensorDataset(torch.Tensor(X_train), torch.LongTensor(y_train))
test_data = TensorDataset(torch.Tensor(X_test), torch.LongTensor(y_test))

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True)

In [315]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size,  num_layers = 2, batch_first=True)
        
        # 출력층
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True)
        

    def forward(self, x):
        x, _staus = self.rnn(x)
        x = self.fc(x)
        
        return x

In [316]:
net = Net(input_size, hidden_size, output_size)

In [317]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [318]:
for epoch in range(epochs):
    for x, y in train_loader:
        optimizer.zero_grad()
        output = net(x)
        loss = criterion(output,y) 
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(loss)

tensor(1.1175, grad_fn=<NllLossBackward0>)
tensor(0.0757, grad_fn=<NllLossBackward0>)
tensor(0.2805, grad_fn=<NllLossBackward0>)
tensor(0.0232, grad_fn=<NllLossBackward0>)
tensor(0.0148, grad_fn=<NllLossBackward0>)
tensor(0.0007, grad_fn=<NllLossBackward0>)
tensor(0.0021, grad_fn=<NllLossBackward0>)
tensor(0.0428, grad_fn=<NllLossBackward0>)
tensor(0.7639, grad_fn=<NllLossBackward0>)
tensor(0.0323, grad_fn=<NllLossBackward0>)


In [325]:
# 모델 평가
net.eval()
correct = 0
total = 0

with torch.no_grad():
    for x, y in test_loader:
        outputs = net(x)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

        
accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.3f}%')

Test Accuracy: 97.778%


tensor([6.1000, 2.8000, 4.7000, 1.2000])

In [358]:
idx = 27

x = test_loader.dataset.tensors[0][idx].view(-1, 4)

y = test_loader.dataset.tensors[1][idx].view(1)

net.eval()
outputs = net(x)

_, predicted = torch.max(outputs.data, 1)

print(predicted, y)

if predicted == y:
    print("correct!")
else:
    print('땡~')

tensor([2]) tensor([2])
correct!


correct!


tensor([1])